In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-opt")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity_projects
Azure region: westeurope
Subscription id: e6a2f2b5-807e-487e-a638-19f0b31e4e2a
Resource group: udacity_labs


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cluster_name = "criscluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Creating Hyperdrive configuration:

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil
from scipy.stats import loguniform
from azureml.train.hyperdrive import BanditPolicy

# Parameter sampler suitable for logistic regression
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,1.0,10),
        '--max_iter': choice(100,200,300)
    }
)


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1,
            evaluation_interval = 1,
            delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

script_folder = "./training"    
    
# Copy the train.py into the script folder
shutil.copy('train.py', script_folder)

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
        script = 'train.py',
        compute_target=cluster_name,
        environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=6,
                                    max_concurrent_runs=4)


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_8ca5fa53-26ed-4773-8937-5f8f843b2301',
 'target': 'criscluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-11T16:27:04.21609Z',
 'endTimeUtc': '2022-03-11T16:47:29.946965Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1c44d8d4-03d4-4ed2-a5c1-04ee0982cbb2',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '9',
  'score': '0.914567526555387',
  'best_child_run_id': 'HD_8ca5fa53-26ed-4773-8937-5f8f843b2301_4',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://udacityproject2455527115.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_8ca5fa53-26ed-4773-8937-5f8f843b2301/azureml-

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [5]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())
print('File name: ', best_run.get_file_names())

ID :  HD_8ca5fa53-26ed-4773-8937-5f8f843b2301_4
Metrics :  {'Regularization Strength:': 1.0, 'Max iterations:': 200, 'Accuracy': 0.914567526555387}
File name:  ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [6]:
best_model = best_run.register_model(model_name = 'hyper_model',model_path = 'outputs/model.joblib')

Preparing data for AutoML run:

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dataset_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path=dataset_url)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
from sklearn.model_selection import train_test_split

ds = x.join(y)

train, test = train_test_split(ds, test_size = 0.2, random_state = 1)

In [10]:
train.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
11703,34,1,0,0,0,5,4,414,2,999,...,1,0,0,0,0,0,1,0,0,0
14460,36,1,0,0,0,5,5,89,2,999,...,1,0,0,1,0,0,0,0,0,0
21996,33,1,0,1,0,5,1,72,4,999,...,0,0,1,0,0,0,0,0,0,0
5973,29,0,0,1,0,6,1,359,1,999,...,1,0,0,0,0,0,0,1,0,0
19045,34,1,0,1,0,8,1,343,2,999,...,0,0,0,0,0,0,0,1,0,0


In [11]:
test.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
6018,58,1,0,0,0,5,5,107,4,999,...,1,0,0,0,0,0,0,1,0,0
23900,46,0,0,1,0,12,1,1472,1,6,...,0,0,0,0,0,0,0,0,1,0
6475,40,1,0,0,0,5,5,97,3,999,...,1,0,0,0,0,0,0,0,1,0
20816,29,1,0,1,0,5,2,383,5,999,...,0,0,1,0,0,0,0,0,0,0
5568,33,0,0,1,0,6,4,155,1,999,...,1,0,0,1,0,0,0,0,0,0


In [12]:
train.to_csv('training/train_ds.csv')

In [13]:
exp_automl = Experiment(workspace=ws, name="automl-exp")
datastore = ws.get_default_datastore()
print(datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-0fadde83-e8d4-4921-aca7-4285445d01b1",
  "account_name": "udacityproject2455527115",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [14]:
datastore.upload(src_dir ='training/', target_path = 'data/')
train_set = TabularDatasetFactory.from_delimited_files(path = [(datastore,('data/train_ds.csv'))])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading training/train.py
Uploaded training/train.py, 1 files out of an estimated total of 2
Uploading training/train_ds.csv
Uploaded training/train_ds.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [15]:
from azureml.train.automl import AutoMLConfig
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = cpu_cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_set,
    label_column_name='y',
    n_cross_validations=5)


In [16]:
automl_run = exp_automl.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on criscluster with default configuration
Running on remote compute: criscluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_f8895636-4f14-42d6-a3f6-3afa8a6ab07f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [17]:
# Submit your automl run
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [18]:
# Get your best run and save the model from that run.
best_run, fitted_model = automl_run.get_output()

print(best_run)
print(fitted_model)

joblib.dump(value=fitted_model, filename='outputs/automl_best_model.pkl')

Run(Experiment: automl-exp,
Id: AutoML_f8895636-4f14-42d6-a3f6-3afa8a6ab07f_30,
Type: azureml.scriptrun,
Status: Completed)
None


['outputs/automl_best_model.pkl']

In [19]:
best_model = best_run.register_model(model_name = 'autoML_model',model_path = './')

In [20]:
#delete compute target

cpu_cluster.delete()